In [1]:
import inro.modeller as m
import os
import glob
import csv

mm = m.Modeller()
eb = m.Modeller().emmebank

#tools
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")           
change_matrix = mm.tool("inro.emme.data.matrix.change_matrix_properties")
create_matrix =  mm.tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = mm.tool("inro.emme.matrix_calculation.matrix_calculator")
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')

#HDR Scripts
common = mm.module('org.emme.common')
moe = mm.module('org.emme.moe')

#scenario folder names
sc_2011 = "2011 GGHM validation run"
sc_2041_base = "2041\\GGHM Base Case"
sc_2041_donothing = "2041\\DoNothing scenario"
sc_2016 = "2016"
sc_2041_C_plus = "2041\\MTO_2041_GGH_MTP_302_V103_CS221_20180601"
sc_2041_C_plus_w_GTAWest = "2041\\MTP_304"
sc_2041_base_new = "2041\\GGHM Base Case\\USE - Sent Through FTP September 20 2018"
sc_2041_C_plus_w_GTAWest_new = "2041\\Committed_Plus"
#sc_2016_new = "2016\\MTO_2016_GGHM_RC61_cs211_GGH_MTP_20171031"

## zone grouping definitions ## 

zones_pd1 = "gf1"
zones_airports = "ga1"
zones_rural = "gb1"
zones_urban = "gb2;gb3;gb4"
zones_ugc = "gb4"
zones_inner = "gc1"
zones_outer = "gc2"
zones_university = "gd1"
zones_employment = "ge1"
zones_hospitals = "gh1"
shed_central = "gs10;gs11"
shed_central_urban = "gs10" # without pd1
shed_central_rural = "gs11"
shed_east = "gs20;gs21"
shed_east_urban = "gs20"
shed_east_rural = "gs21"
shed_midwest = "gs30;gs31"
shed_midwest_urban = "gs30"
shed_midwest_rural = "gs31"
shed_southwest = "gs40;gs41"
shed_southwest_urban = "gs40"
shed_southwest_rural = "gs41"
shed_west = "gs50;gs51"
shed_west_urban = "gs50"
shed_west_rural = "gs51"
shed_north = "gs60;gs61"
shed_north_urban = "gs60"
shed_north_rural = "gs61"
shed_niagara = "gs70;gs71"
shed_niagara_urban = "gs70"
shed_niagara_rural = "gs71"
shed_urban = "gs10;gs20;gs30;gs40;gs50;gs60;gs70"
shed_rural = "gs11;gs21;gs31;gs41;gs51;gs61;gs71"


In [2]:
### 2041 Base
sc_num_am = 103
sc_num_md = 104
scenario_folder = sc_2041_base

###2016
#sc_num_am = 109
#sc_num_md = 110
#scenario_folder = sc_2016

#scenarios
sc_am = eb.scenario(sc_num_am)
sc_md = eb.scenario(sc_num_md)

#networks
ntwk_am = sc_am.get_network()
ntwk_md = sc_md.get_network()

#input folder
matrix_folder = u"P:\\451010\\281829_GGH\\GGHM Runs\\" + scenario_folder

In [13]:
#delete all FULL matrices
common.clear_matrices(eb)

In [14]:
# distance
sov_dist_am = "skim.am_peak.auto.sov.dist"
trips_all_daily = "trips_all_modes_daily"

matrix_list ={
    trips_all_daily :"",
    sov_dist_am :""
    }

#Import matrices
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name,
                  matrix_description = matrix_name)
    
common.import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am) 

Imported matrix mf2
Imported matrix mf1
Finished Importing Matrices


In [17]:

zones_array = [zones_pd1,\
               shed_central,\
               shed_east,\
               shed_midwest,\
               shed_southwest,\
               shed_west,\
               shed_north,\
               shed_niagara]

In [18]:
for shed in zones_array:
    print shed
    print "origins"
    spec = common.get_mc_spec(None, matrix_list[trips_all_daily],"+","+",shed,"all",None)
    all_trips = matrix_calculator(spec)['result']
    spec ['expression'] = matrix_list[trips_all_daily] + "*" + matrix_list[sov_dist_am]
    all_trip_length = matrix_calculator(spec)['result']
    print all_trip_length / all_trips
    print "destinations"
    spec = common.get_mc_spec(None, matrix_list[trips_all_daily],"+","+","all",shed,None)
    all_trips = matrix_calculator(spec)['result']
    spec ['expression'] = matrix_list[trips_all_daily] + "*" + matrix_list[sov_dist_am]
    all_trip_length = matrix_calculator(spec)['result']
    print all_trip_length / all_trips

gf1
origins
3.67961333667
destinations
10.9823372853
gs10;gs11
origins
8.13766035986
destinations
9.44265167767
gs20;gs21
origins
13.8816768361
destinations
9.54602993321
gs30;gs31
origins
9.67336101827
destinations
9.8650151096
gs40;gs41
origins
10.1144231081
destinations
9.30685640156
gs50;gs51
origins
9.72839025112
destinations
10.3981720448
gs60;gs61
origins
14.4276957883
destinations
10.919528416
gs70;gs71
origins
10.6154284147
destinations
8.48936101909


In [48]:
#import distance matrix - use same for all scenarios

common.clear_matrices(eb)

dist_folder = u"P:\\451010\\281829_GGH\\GGHM Runs\\2041\\GGHM Base Case\\Sent Through FTP in 2017\\2041_sov_am_peak_skims\\"
sov_dist_am = "mf1"
matrix_list = {
    "skim.am_peak.auto.sov.dist": sov_dist_am
    }

common.import_matrices_from_directory(dist_folder, matrix_list, ".mdf",103) 

Imported matrix mf1
Finished Importing Matrices


In [46]:
### 2041 Base
#sc_num_am = 103
#scenario_folder = sc_2041_base_new

###2016
sc_num_am = 109
scenario_folder = sc_2016

#scenarios
sc_am = eb.scenario(sc_num_am)

#networks
ntwk_am = sc_am.get_network()

#input folder
matrix_folder = u"P:\\451010\\281829_GGH\\GGHM Runs\\" + scenario_folder

In [47]:
#Mode Split By Distance 
sov_dist_am = "mf1"
common.clear_matrices(eb,[sov_dist_am])

# distance
trips_active = "trips_active_daily"
trips_auto= "trips_auto_daily"
trips_transit = "trips_transit_daily"

matrix_list ={
    trips_auto :"",
    trips_transit :"",
    trips_active :""
    }

#Import matrices
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = matrix_list[matrix_name],
                matrix_name= matrix_name,
                matrix_description = matrix_name)
    
common.import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am) 

Imported matrix mf3
Imported matrix mf2
Imported matrix mf4
Finished Importing Matrices


In [49]:
#divide 2016 auto trips in half
spec = {
    "expression": matrix_list[trips_auto] + "/2",
    "result": matrix_list[trips_auto],
    "constraint": {
        "by_value": None,
        "by_zone": None
    },
    "aggregation": {
        "origins": None,
        "destinations": None
    },
    "type": "MATRIX_CALCULATION"
}

matrix_calculator(spec)

{'average': 1.573828,
 'expression_type': 'FULL',
 'maximum': 20818.78125,
 'maximum_at': {'destination': 8885, 'origin': 8885},
 'minimum': 0.0,
 'minimum_at': {'destination': 1034, 'origin': 1001},
 'num_evaluations': 12138256,
 'result_type': 'FULL',
 'sum': 19103524.0}

In [37]:
dest = {
    "UGC":"gb4",
    "Meadowvale": "ge6",
    "Sheridan": "ge7",
    "Mississauga UGC": "ge13",
    "Brampton UGC": "ge14",
    "YongeEg" : "ge15",
    "YongeShepp" : "ge16",
    "STC" : "ge18",
    "Toronto East" : "ge3",
    "York East" : "ge5",
    "Toronto West" : "ge2",
    "York West" : "ge4",
    "PD1" : "gf1",
    "Pickering UGC": "2015;2016;2019",
    "Oshawa UGC": "2132;2136;2040",
    "St Catharines UGC" : "7114;7115;7120;7121;7124;7131",
    "Barrie UGC" : "8845;8848;8853;8856",
    "Newmarket UGC" : "3412;3413;3421",
    "Hamilton" : "ge12",
    "Guelph UGC" : "8542;8545;8547;8548;8550;8551;8554;8557;8558;8559;8561;8563",
    "Waterloo" : "ge10",
    "Kitchener": "ge11"
    }

In [80]:

spec = common.get_mc_spec(None, matrix_list[trips_active],"+","+","all",dest["Kitchener"],None)
print matrix_calculator(spec)['result']
spec['constraint']['by_value']= {}
spec['constraint']['by_value']['od_values'] = sov_dist_am
spec['constraint']['by_value']['condition'] = 'INCLUDE'

for dist in range(1,50):
    spec['constraint']['by_value']['interval_min'] = dist-1
    spec['constraint']['by_value']['interval_max'] = dist
    print matrix_calculator(spec)['result']

2515.203
741.5617
651.457
413.0958
255.5842
154.4636
145.5719
76.81611
35.06624
20.53931
8.34692
5.676951
1.773435
0.757155
0.501437
0.184532
1.26291
0.367306
0.116377
0.474634
0.290869
0.218655
0.21852
0.178477
0.24193
0.132766
0.133354
0.054752
0.006329
0.06185
0.012526
0.003918
0.014636
0.006583
0.003292
0.0004245335
0.0004954635
0.001433
0.002026
0.002294
0.00020652
8.563666e-08
3.003126e-07
1.779141e-06
1.092706e-06
0.0001824208
0.0001853197
0.000398083
6.356868e-06
1.454636e-05


In [22]:
matrix_list[sov_dist_am]

'mf4'